# LLama 2


Llama2는 Meta AI에서 개발한 강력하고 유연한 모델로, 고품질 텍스트를 생성할 수 있습니다. 이 모델은 다른 모델과는 차별화되는 여러 가지 강점을 가지고 있습니다. 먼저, Llama2는 기사, 이야기, 챗봇 대화 등 다양한 스타일과 형식의 텍스트를 생성할 수 있습니다. 둘째, 논리적이고 유창하며 자연스러운 흐름과 구조를 가진 텍스트를 생성할 수 있습니다. 마지막으로, Llama2는 높은 확장성을 가지고 있어 큰 양의 텍스트를 빠르고 효율적으로 생성하는 데 사용될 수 있습니다. 전반적으로, Llama2는 다양한 형식의 고품질 텍스트를 생성하려는 누구에게나 가치 있는 도구입니다.<br>해당 모델을 사용하기 위해서는메타 홈페이지를 통해서 라이센스 및 토큰을 흭득한 후 이용할 수 있습니다.


# Google Colab +
해당 모델은 Google의 클라우드 시스템중 하나인 Colab을 이용하여 작성 하였습니다 일반 모델의 경우 사양이 낮기 때문에 LLM을 구동하기 적절치 않아 프로 서비스를 구독하여 작성 됐습니다.

## Step 1. 필수 Library다운 및 허깅페이스 로그인

In [ ]:
# 사양 체크(그래픽 카드)
!nvidia-smi

Sun Sep  3 06:27:29 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    25W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# 사양 체크(램)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip install -q datasets loralib sentencepiece
!pip -q install bitsandbytes accelerate xformers einops
!pip -q install langchain
!pip -q install accelerate

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 80.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.0/167.0 MB 6

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## Step2. 라이브러리 불러오기 및 모델 설정

In [ ]:
from huggingface_hub import hf_hub_download
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

from langchain.llms import LlamaCpp
from langchain import PromptTemplate, LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
from langchain import HuggingFacePipeline
from langchain import PromptTemplate,  LLMChain

import json
import textwrap


from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain
from langchain.chains import SequentialChain
from langchain.document_loaders import PyPDFLoader

import warnings
warnings.filterwarnings('ignore')

In [ ]:
#모델 다운로드

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                          use_auth_token=True,)

model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             use_auth_token=True,
                                             load_in_4bit=True
                                             )

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Use a pipeline for later
from transformers import pipeline

pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 4096,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

In [ ]:
llm = HuggingFacePipeline(pipeline = pipe, model_kwargs = {"max_new_tokens": 4096, 'temperature':0.3})

## Step 3. 템플릿 작성

In [ ]:
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = """\You are a helpful, respectful and honest assistant. \
Always answer as helpfully as possible, while being safe.\
Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.\
Please ensure\ that your responses are socially unbiased and positive in nature.\
If a question does not make any sense\, or is not factually coherent\, explain why instead of answering something not correct.\
If you don't know the answer to a question\, please don't share false information.\
"""
default_input_variables = []


def get_prompt(instruction, new_system_prompt = DEFAULT_SYSTEM_PROMPT, input_variables = default_input_variables):
    SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
    prompt_schema =  B_INST + SYSTEM_PROMPT + instruction + E_INST
    prompt_template = PromptTemplate(template = prompt_schema, input_variables = input_variables)
    return prompt_template

def cut_off_text(text, prompt):
    cutoff_phrase = prompt
    index = text.find(cutoff_phrase)
    if index != -1:
        return text[:index]
    else:
        return text

def remove_substring(string, substring):
    return string.replace(substring, "")

def generate(text):
    outputs = model(text)
    final_outputs = cut_off_text(outputs, '</s>')
    final_outputs = remove_substring(final_outputs, text)

    return final_outputs#, outputs

def parse_text(text):
        wrapped_text = textwrap.fill(text, width=100)
        print(wrapped_text +'\n\n')
        # return assistant_text


In [ ]:
analyst = """"You are a very talanted company analyst.\
You are great at answering questions about analyizing companies in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.
"""

first_instruction_template = """Analzye the company named {com_name} \
and wirte down the short report based on the \
analysis of the company with its introduction, business and characteristics delimited by triple backticks \
into a style that is {style}. \
\n
Company:
```Company Introduction: {introduction}``` \
\n
```Comapny Business: {business}``` \
\n
```Comapny Characteristic:{characteristic}```
"""

company_name = "LG Energy"

style = """Profssional \
English \
but easy to transalte"""


introduction =  """
더 나은 세상을 위해 에너지에 대한 다양한 솔루션을 제공합니다.
"""

business = """
- 당사는 1992년 리튬이온전지 연구를 착수하고 1996년에는 리튬이온전지 개발을 시작하였습니다.
EV 시장의 시작부터 함께하며 시장을 선도하고 있으며 지속적인 성장을 거듭하며 끊임없는 연구개발과 신제품 출시 및 품질 혁신을 통해 한국의 에너지솔루션 산업을 대표하는 글로벌 기업으로 성장하였습니다.
국내는 물론 세계 주요 거점에 생산, 판매, R&D 네트워크를 구축하고 전세계를 대상으로 사업을 전개하고 있습니다.
향후 폭발적으로 성장하는 배터리 시장에서 선두 기업으로 영속하기 위해 사업의 내실을 다지고 품질, 미래준비,Sustainability 추진을 지속해 나가도록 하겠습니다.
당사는 현재 EV, ESS, IT기기, 전동공구, LEV 등에 적용되는 전지 관련 제품의 연구, 개발,제조, 판매하는 사업을 영위하고 있으며 에너지솔루션 단일 사업부문으로 구성되어 있습니다.
2022년 연간 기준으로 약 25조 5,986억원의 매출과 약 1조 2,137억원의 영업이익을 달성하였습니다.
또한 당사는 본사의 공장 및 해외 생산법인에서 제품을 생산하며, 제품 판매는 본사 및 해외 생산법인 또는 해외 판매법인을 통해 운영하고 있으며 대부분의 매출은 본사 및 해외 생산법인의 직접 판매로부터 발생하고 있습니다.
당사는 리튬전지 시장에서 확고한 일등 지위 달성을 위해 집중적인 R&D 투자를 통해 고객/포트폴리오별 최적의 솔루션을 도출하고, 안전성강화 등 품질 향상 및 글로벌 Operation 경쟁력을 확보하여 고객에게 안정적으로 물량 공급을 지속하고자 합니다.
"""

characteristic = """
1. 경영 이념- LG는 ‘고객을 위한 가치창조’와 ‘인간존중의 경영’을 경영이념으로 공유하고, 경영현장의 정신에 따라 자율과 책임에 기초한 자율경영을 도모합니다.
자유롭고 공정한 경쟁을 지향하는 자유시장경제 질서를 존중하고, 상호신뢰와 협력을 토대로 모든 이해관계자와 공동의 이익을 추구함으로써 세계적인 초우량 기업으로 영속 발전하기 위해 우리는 모든 임직원이 지켜야 할 올바른 행동과 가치판단의 기준으로서 윤리규범을 제정하고 그 실천을 다짐합니다.

2. 행동 방식 - 정도 경영
정도경영은 윤리경영을 기반으로 꾸준히 실력을 배양해 정정당당하게 승부하는 LG만의 행동방식입니다.
LG 고유의 기업문화인 LG Way는 경영이념인 “고객을 위한 가치창조”와 “인간존중의 경영”을 LG 행동방식인 정도경영으로 실천함으로써 궁극적 지향점인 “일등 LG”를 달성하자는 것을 의미합니다. LG가 추구하는 근본적인 가치 즉, 경영이념을 실천하는 LG 고유의 행동방식이 바로 정도경영입니다.
정도경영이란 단순히 윤리경영만을 의미하는 것이 아니며, 진정한 의미의 정도경영은 윤리경영에서 나아가 경쟁에서 이길 수 있는 실력을 바탕으로 실질적인 성과를 창출하는 것을 의미합니다.
- 정직
원칙과 기준에 따라 투명하게 일한다.
- 공정한 대우
모든 거래관계에서 공평하게 기회를 제공하고 공정하게 대우한다
- 실력을 통한 정당한 경쟁
정정당당하게 경쟁하여 이길 수 있는 실력을 키운다.

3. 비전 - 일등 LG
LG의 궁극적인 지향점으로 시장에서 인정받으며 시장을 리드하는 선도기업이 되는 것
"""

first_prompt = get_prompt(instruction = first_instruction_template, new_system_prompt = analyst,
                    input_variables = ["com_name", "style", "introduction", "business", "characteristic"])
analyze_company = first_prompt.format(com_name = company_name, style = style, introduction =  introduction,
                             business = business, characteristic = characteristic)

print(analyze_company)

[INST]<<SYS>>
"You are a very talanted company analyst.You are great at answering questions about analyizing companies in a conciseand easy to understand manner. When you don't know the answer to a question you admitthat you don't know.

<</SYS>>

Analzye the company named LG Energy and wirte down the short report based on the analysis of the company with its introduction, business and characteristics delimited by triple backticks into a style that is Profssional English but easy to transalte. 

Company:
```Company Introduction: 
더 나은 세상을 위해 에너지에 대한 다양한 솔루션을 제공합니다.
``` 

```Comapny Business: 
- 당사는 1992년 리튬이온전지 연구를 착수하고 1996년에는 리튬이온전지 개발을 시작하였습니다. 
EV 시장의 시작부터 함께하며 시장을 선도하고 있으며 지속적인 성장을 거듭하며 끊임없는 연구개발과 신제품 출시 및 품질 혁신을 통해 한국의 에너지솔루션 산업을 대표하는 글로벌 기업으로 성장하였습니다. 
국내는 물론 세계 주요 거점에 생산, 판매, R&D 네트워크를 구축하고 전세계를 대상으로 사업을 전개하고 있습니다. 
향후 폭발적으로 성장하는 배터리 시장에서 선두 기업으로 영속하기 위해 사업의 내실을 다지고 품질, 미래준비,Sustainability 추진을 지속해 나가도록 하겠습니다.
당사는 현재 EV, ESS, IT기기, 전동공구, LEV 등에 적용되는 전지 관련 제품의 연구, 개발,제조, 판매하는 사업을

In [ ]:
analyst_candidate = """"You are a very job consultant.\
You are great at analyizing companies's job description \
and understand what hr manager wants for candidates. \
When you don't know the answer to a question you admit \
that you don't know.
"""

second_instruction_template = """Wrtie the key points to pass the job posted by company named {com_name} \
based on analysis of the job description \
with its Task, eligibility and Preferential delimited by triple backticks \

Job description:
```Type of talent: {type_talent}``` \

```Position Name: {pos_name}``` \
\n
```Task: {task}``` \
\n
```Eligibility: {eligibility}``` \
\n
```Preferential:{preferential}`` \
\n
into a style that is {style}.

Please refer to the report of the company named {com_name} analyze_company.
"""

company_name = company_name

type_talent = """"
1. 열정
‘처음’이 되고 싶은 열망으로 세상의 혁신을 주도한다. 누가 시키지 않더라도 자발적인 의지로 변화를 만들어낸다.

2. 전문성
겸손한 자세로 실력을 쌓아 세계 최고의 솔루션을 제공한다. 가보지 않은 길을 향해 끊임없이 도전하며 집요하게 실행한다.

3. 팀워크
남에게 미루지 않고 기꺼이 내가 먼저 시작한다. 경계를 넘나들며 우리의 목표를 함께 이룬다
"""

pos_name = """Chief Data Officer - 기획"""

task = """
조직별 목표를 관리하고 그 목표를 수월하게 달성할 수 있도록 기반이 되는 모든 사항을 지원해요.
이를 위해 효율적인 조직관리를 위한 업무를 기획/운영하고 보통 목표 관리/투자 및 예산 관리/HR관리/조직문화 개선 등의 업무를 수행해요
"""

eligibility = """
AI/빅데이터 등 DX 조직을 운영 및 관리하는 업무이므로 DX 관련된 지식 및 관심도가 높아야해요.
또한 다양한 조직과 협업해야하는 위치인만큼 주도적인 성격을 갖춘 분이면 좋아요.
사람들과 대화하는 것에 거리낌이 없고, 커뮤니케이션 능력이 뛰어나다면 금상첨화!
"""

preferential ="""
DX 관련 자격증을 소유한 분이면 좋아요.
또한 글로벌 전 지역에 사업장을 보유한 회사인만큼, 외국어 능력을 갖추고 있어야 합니다. 가장 중요한 언어는 영어입니다!

가장 중요한 것은 DX에 대한 관심이에요. CDO 조직은 회사 내 제조/비제조 등 다양한 영역에서 주도적으로 DX화를 추진하여 회사의 효율성 및 이익을 극대화하는 조직이에요.
빠르게 성장할 조직이며, 해당 조직에서의 성장을 토대로 배터리 업계 DX 전문가로 성장하겠다는 포부를 드러내면 좋아요.
"""

style = "Easy to translate \
professional\
English"

# analyze_company = mobis


second_prompt = get_prompt(instruction = second_instruction_template, new_system_prompt = analyst_candidate,
                    input_variables = ["com_name", "style", "type_talent", "pos_name", "task", "eligibility",
                                      "preferential"])
hiring_company = second_prompt.format(com_name = company_name, style = style, type_talent = type_talent,
                                     pos_name = pos_name, task = task, eligibility = eligibility,
                                      preferential = preferential)

print(hiring_company)

[INST]<<SYS>>
"You are a very job consultant.You are great at analyizing companies's job description and understand what hr manager wants for candidates. When you don't know the answer to a question you admit that you don't know.

<</SYS>>

Wrtie the key points to pass the job posted by company named LG Energy based on analysis of the job description with its Task, eligibility and Preferential delimited by triple backticks 
Job description:
```Type of talent: "
1. 열정
‘처음’이 되고 싶은 열망으로 세상의 혁신을 주도한다. 누가 시키지 않더라도 자발적인 의지로 변화를 만들어낸다.

2. 전문성
겸손한 자세로 실력을 쌓아 세계 최고의 솔루션을 제공한다. 가보지 않은 길을 향해 끊임없이 도전하며 집요하게 실행한다.

3. 팀워크
남에게 미루지 않고 기꺼이 내가 먼저 시작한다. 경계를 넘나들며 우리의 목표를 함께 이룬다
``` 
```Position Name: Chief Data Officer - 기획``` 

```Task: 
조직별 목표를 관리하고 그 목표를 수월하게 달성할 수 있도록 기반이 되는 모든 사항을 지원해요. 
이를 위해 효율적인 조직관리를 위한 업무를 기획/운영하고 보통 목표 관리/투자 및 예산 관리/HR관리/조직문화 개선 등의 업무를 수행해요
``` 

```Eligibility: 
AI/빅데이터 등 DX 조직을 운영 및 관리하는 업무이므로 DX 관련된 지식 및 관심도가 높아야해요.
또한 다양한 조직과 협업해야하는 위치인만큼 주도적인 성격을 갖춘 분이면 좋아요.
사람들과 대화하는 것에

In [ ]:
# 체인으로 윗 두스텝 합치기
prompt_template_one = first_prompt
company_report = LLMChain(llm=llm, prompt=prompt_template_one, output_key = "analyze_company")

prompt_template_two = second_prompt
job_report = LLMChain(llm=llm, prompt=prompt_template_two, output_key = "job_description_report")

first_chain = SequentialChain(chains=[company_report, job_report],
                              input_variables = ["com_name", "style", "introduction", "business", "characteristic",
                                                 "type_talent", "pos_name", "task", "eligibility", "preferential"],
                              output_variables = ["analyze_company", "job_description_report"])

company_report = first_chain({"com_name" : company_name, "style" : style, "type_talent" : type_talent, "characteristic" : characteristic, "business" : business, "introduction": introduction,
                                     "pos_name" : pos_name, "task" : task, "eligibility" : eligibility, "preferential" : preferential})

In [ ]:
print(company_report["analyze_company"])

  Based on the information provided, LG Energy is a company that specializes in providing various energy solutions, including batteries, energy storage systems, and electric vehicle (EV) charging infrastructure. The company has been growing rapidly since its establishment in 1992 and has become a global leader in the energy industry.
Here are some key characteristics of LG Energy:
1. Management Philosophy: LG Energy's management philosophy is centered around creating value for customers and stakeholders through ethical and responsible business practices. The company aims to be a leading player in the energy industry by consistently delivering high-quality products and services.
2. Business Style: LG Energy's business style is based on the concept of "win-win" competition, where the company seeks to create mutually beneficial relationships with its customers, suppliers, and partners. This approach is reflected in the company's commitment to fairness, transparency, and mutual respect.
3.

In [ ]:
print(company_report["job_description_report"])

  Based on the job description provided, here are the key points that could help you pass the job posted by LG Energy:
1. Strong passion for DX (Digital Transformation) and willingness to drive innovation and change within the organization.
2. Excellent communication and interpersonal skills, with the ability to connect with people at all levels of the organization.
3. Proficiency in global languages, with a focus on English, as it is the most important language in the organization.
4. Strong interest in DX and willingness to grow as a DX expert within the organization.
5. Ability to work independently and take initiative, with a proactive and results-driven mindset.
6. Experience in managing and analyzing data to make informed decisions and drive business growth.
7. Familiarity with global business practices and cultures, with the ability to work effectively in a diverse and fast-paced environment.
8. Strong analytical and problem-solving skills, with the ability to identify and prior

In [ ]:
hiring_manager = """"You are a very talanted hiring manager. \
You can determine if someone is good candidate,\
and have a good understanding of the candidate's capabilities, \
strengths, \
and points to be fixed. \
When you don't know the answer to a question you admit \
that you don't know.
"""

fourth_instruction_template = """Write a talent report for the candidate, {can_name}, \
evaluating their suitability for the {pos_name} role \
in the form of report. \
Include their resume enclosed within triple backticks:

Resume: ```{resume}```

Ensure you consider the company report and the key hiring criteria for the {com_name} job posting.
"""

pos_name = pos_name

can_name = "Kim JinGu"

resume = """

Self Introduction:
I am a Data Analyst, have a year of experience in data processing, utilizing machine learning techniques, NLP and leveraging Excel functions.
I have a proven track record of delivering actionable insights and innovative solutions to complex business problems.
I am driven by a passion for continuous improvement and eagerly embrace opportunities to learn new technologies and tools that enhance data-driven decision making.
\n

Education

XI’AN JIAOTONG UNIVERSITY
Master, Management Science and Engineering
2021.09 – 2023.06\
\n
AJOU UNIVERSITY
Bachelor, Business Administration
2012.03 – 2020.02\
\n
CAPITAL UNIVERSITY OF ECONOMICS AND BUSINESS
Chinese Language School
2018.09 – 2019.06
\n


Experiences 1:
Company Name: MIDAS IT
Position: HUMAN RESOURCES ANALYST
Period: 2021.03 – 2021.07
1. Designed and implemented an automated data processing pipeline using Python to extract and transform data from various formats such as Word, PDF, and Excel to a standardized CSV format.
2. Developed a semi-supervised labeling algorithm utilizing machine learning techniques to analyze and identify communication patterns between team leaders and members.
3. Leveraged Excel functions and data manipulation techniques to create a unified and structured table from diverse HR data from multiple teams.

Experiences 2:
Company Name: EDM&Co
Position EVENT OPERATOR
Period: 2019.10 – 2020.09
Wokrs:
1) Collaborated with global event organizers INSOMNIAC and ALDA to successfully organize U.S. franchise festivals, Escape Korea and EDC Korea, attracting over 30,000 attendees per day.
2) Contributed to event planning and execution by analyzing previous event data using advanced statistical techniques and visualization tools.
3) Played a crucial role in supporting the ticket sales team by effectively communicating with foreign ticket vendors, including Klook, Festicket, and Kkday.
4) Provided comprehensive reports on overseas buyers' preferences and behaviors, enabling the team to tailor marketing strategies and optimize ticket sales

Project 1:
Master Degree thesis
Works: A Study on the Consumer Response to VR Content through the VR Concert Comments on YouTube
1. Data Gathering
-  Use Beautiful Soup and Selenium package in order to collects a large amount of YouTube video comments using the .
2. Data Preprocessing
- Basic preprocessing lines such as sewing data and duplicate data processing to improve data quality.
- Compose of an analytical token by performing additional preprocessing for unstructured data, especially for the Short Text Analysis.
3. Data Interpretation
- Perform descriptive analysis of collected data using visualizations
- Analyze the sentiment, the main words of comments using TF-IDF word embeddings.
- Public Opinion analysis by extracting the main topics of more than 30,000 comments using BERTopic.
- Text Regression is performed in combination with numerical data such as number and creation date, and empirical analysis is performed with the comments categorized through topic modeling.

"""

com_name = "LG Energy"


fourth = get_prompt(instruction = fourth_instruction_template, new_system_prompt = hiring_manager,
                    input_variables = ["can_name", "pos_name", "resume", "com_name"])

hr = fourth.format(com_name = com_name, pos_name = pos_name, can_name = can_name, resume = resume)

print(hr)

[INST]<<SYS>>
"You are a very talanted hiring manager. You can determine if someone is good candidate,and have a good understanding of the candidate's capabilities, strengths, and points to be fixed. When you don't know the answer to a question you admit that you don't know.

<</SYS>>

Write a talent report for the candidate, Kim JinGu, evaluating their suitability for the Chief Data Officer - 기획 role in the form of report. Include their resume enclosed within triple backticks:

Resume: ```

Self Introduction:
I am a Data Analyst, have a year of experience in data processing, utilizing machine learning techniques, NLP and leveraging Excel functions.
I have a proven track record of delivering actionable insights and innovative solutions to complex business problems.
I am driven by a passion for continuous improvement and eagerly embrace opportunities to learn new technologies and tools that enhance data-driven decision making.



Education

XI’AN JIAOTONG UNIVERSITY
Master, Management S

In [ ]:
position_manager = """"You are a very talanted boss from the {pos_industry} industry. \
You can determine if someone is good candidate,\
and if somone is proper for your team. \
When you don't know the answer to a question you admit \
that you don't know.
"""

fifth_instruction_template = """Write a talent report for the candidate, {can_name}, \
evaluating their suitability for the {pos_name} role \
in the form of report. \
Include their resume enclosed within triple backticks:

Resume: ```{resume}```

Ensure you consider the company report and the key hiring criteria for the {com_name} job posting.
"""

pos_name = pos_name

can_name = can_name

resume = resume

pos_industry = "IT and Digital"

com_name = com_name

fifth = get_prompt(instruction = fifth_instruction_template, new_system_prompt = position_manager,
                    input_variables = ["pos_industry", "can_name", "pos_name", "resume", "com_name"])

position_manager = fifth.format(pos_industry = pos_industry, com_name = com_name, pos_name = pos_name, can_name = can_name,resume = resume)

print(position_manager)

[INST]<<SYS>>
"You are a very talanted boss from the IT and Digital industry. You can determine if someone is good candidate,and if somone is proper for your team. When you don't know the answer to a question you admit that you don't know.

<</SYS>>

Write a talent report for the candidate, Kim JinGu, evaluating their suitability for the Chief Data Officer - 기획 role in the form of report. Include their resume enclosed within triple backticks:

Resume: ```

Self Introduction:
I am a Data Analyst, have a year of experience in data processing, utilizing machine learning techniques, NLP and leveraging Excel functions.
I have a proven track record of delivering actionable insights and innovative solutions to complex business problems.
I am driven by a passion for continuous improvement and eagerly embrace opportunities to learn new technologies and tools that enhance data-driven decision making.



Education

XI’AN JIAOTONG UNIVERSITY
Master, Management Science and Engineering
2021.09 – 202

In [ ]:
recruiter = """"You are a very talanted recruiter. \
You can determine if someone is good candidate,\
and if somone is proper for your team, \
depend on the report from other managers. \
When you don't know the answer to a question you admit \
that you don't know.
"""

sixth_instruction_template ="""Generate a comprehensive talent report for the candidate, {can_name}, \
referring input from the talent reports of the hiring manager and position manager at {com_name}, \
along with the company report containing key hiring criteria for the {pos_name} of {com_name}
in the form of report. \

Emphasize the candidate's strengths and weaknesses, \
and offer a detailed explanation of their suitability for the {pos_name} role.\

Please ensure the candidate's resume is enclosed within triple backticks:

Resume: ```{resume}```
"""

can_name = "Kim JinGu"

com_name = com_name

pos_name = pos_name

resume = resume

sixth = get_prompt(instruction = sixth_instruction_template, new_system_prompt = recruiter,
                    input_variables = ["can_name", "com_name", "pos_name", "resume"])

recruiter = sixth.format(can_name = can_name, com_name = com_name, pos_name = pos_name, resume = resume)

print(recruiter)

[INST]<<SYS>>
"You are a very talanted recruiter. You can determine if someone is good candidate,and if somone is proper for your team, depend on the report from other managers. When you don't know the answer to a question you admit that you don't know.

<</SYS>>

Generate a comprehensive talent report for the candidate, Kim JinGu, referring input from the talent reports of the hiring manager and position manager at LG Energy, along with the company report containing key hiring criteria for the Chief Data Officer - 기획 of LG Energy
in the form of report. 
Emphasize the candidate's strengths and weaknesses, and offer a detailed explanation of their suitability for the Chief Data Officer - 기획 role.
Please ensure the candidate's resume is enclosed within triple backticks:

Resume: ```

Self Introduction:
I am a Data Analyst, have a year of experience in data processing, utilizing machine learning techniques, NLP and leveraging Excel functions.
I have a proven track record of delivering act

In [ ]:
## Manger들의 query를 체인으로 연결(프롬프트 4~6)

prompt_template_four = fourth
hr_report = LLMChain(llm=llm, prompt=prompt_template_four, output_key = "hr_report")

prompt_template_five = fifth
manager_report = LLMChain(llm=llm, prompt=prompt_template_five, output_key = "manager_report")

prompt_template_six = sixth
final_report = LLMChain(llm=llm, prompt=prompt_template_six, output_key = "final_report")

second_chain = SequentialChain(chains=[hr_report, manager_report,final_report],
                              input_variables = ["can_name", "pos_name", "resume", "com_name",  "pos_industry"],
                              output_variables = ["hr_report", "manager_report", "final_report"])

talent_report = second_chain({"can_name": can_name, "com_name" : com_name, "pos_name": pos_name, "resume": resume, "pos_industry": pos_industry})

This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (4096). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


In [ ]:
print(talent_report["hr_report"])

In [ ]:
print(talent_report["manager_report"])

  Subject: Talent Report for Kim JinGu - Chief Data Officer Candidate
Dear [Hiring Manager],
I am writing to provide a talent report for Kim JinGu, who has been identified as a potential candidate for the Chief Data Officer position at LG Energy. As his former supervisor, I have had the pleasure of observing and guiding him throughout his professional journey, and I believe he possesses the skills and qualities necessary to excel in this role.
Professional Experience:
Kim JinGu has an impressive track record of delivering data-driven solutions to complex business problems. His experience in data processing, utilizing machine learning techniques, and leveraging Excel functions has enabled him to provide actionable insights and innovative solutions to his previous employers. He has demonstrated a proven ability to analyze and interpret large datasets, identify patterns and trends, and communicate his findings effectively to both technical and non-technical audiences.
Key Strengths:
1. Da

In [ ]:
print(talent_report["final_report"])

  Sure, here's the comprehensive talent report for Kim JinGu, based on the input from the talent reports of the hiring manager and position manager at LG Energy, along with the company report containing key hiring criteria for the Chief Data Officer - 기획 role:
Talent Report for Kim JinGu
Overall Assessment: Strong candidate with a unique blend of technical, business, and interpersonal skills, making him an ideal fit for the Chief Data Officer - 기획 role at LG Energy.
Technical Skills:
* Proficient in data processing, utilizing machine learning techniques, NLP, and Excel functions
* Demonstrated ability to design and implement automated data processing pipelines using Python
* Familiarity with statistical techniques and data visualization tools

Business Skills:
* Strong understanding of business operations and strategic planning
* Proven track record of delivering actionable insights and innovative solutions to complex business problems
* Adept at leveraging data-driven decision making 

In [ ]:
candidate = """
You are a talanted job candidate \
proficient in crafting compelling appeals to hiring managers. \
Your expertise lies in adeptly melding company requirements \
with your client's narrative. \
When you don't know the answer to a question you admit \
that you don't know.
"""

sevnth_instruction_template = """You have to answer the question of the hiring manager. \
Write a narrative sentence to \
the question of hiring manager within triple backticks: \
```Question: {question}```

In your reply, recount the candidate's resume,
{resume}, \
and integrate candidate's essential points, {point}. \

Be sure to refer the analyze_company \
and the job_description_report \
of the posted position named {pos_name} of {com_name}\
and the final_report of {can_name}. \

Keep the response length at approximately {num_bytes} bytes.
"""

question = "Please introduce your motivation for applying for LG Energy Solution and recruitment in detail."

can_name = can_name

resume = resume

point = """1. Have expereience of skills related to the position
 """

pos_name = pos_name

com_name = com_name

num_bytes = "1200"


sevntth_prompt = get_prompt(instruction = sevnth_instruction_template, new_system_prompt = candidate,
                    input_variables = ["question", "can_name", "resume", "point", "pos_name", "num_bytes", "com_name"])

answer_1 = sevntth_prompt.format(question = question, can_name = can_name, resume = resume, point = point, num_bytes = num_bytes,
                                pos_name = pos_name, com_name = com_name)

print(answer_1)

[INST]<<SYS>>

You are a talanted job candidate proficient in crafting compelling appeals to hiring managers. Your expertise lies in adeptly melding company requirements with your client's narrative. When you don't know the answer to a question you admit that you don't know.

<</SYS>>

You have to answer the question of the hiring manager. Write a narrative sentence to the question of hiring manager within triple backticks: ```Question: Please introduce your motivation for applying for LG Energy Solution and recruitment in detail.```

In your reply, recount the candidate's resume,


Self Introduction:
I am a Data Analyst, have a year of experience in data processing, utilizing machine learning techniques, NLP and leveraging Excel functions.
I have a proven track record of delivering actionable insights and innovative solutions to complex business problems.
I am driven by a passion for continuous improvement and eagerly embrace opportunities to learn new technologies and tools that enha

In [ ]:
ans_1_chain = LLMChain(llm=llm, prompt = sevntth_prompt)
answer_1 = ans_1_chain.run({"question" : question, "can_name" : can_name, "resume": resume, "point" : point, "num_bytes" : num_bytes,
                                "pos_name" : pos_name, "com_name" : com_name})
print(answer_1)

  Dear [Hiring Manager],
I am excited to apply for the Chief Data Officer position at LG Energy Solution, as it aligns with my passion for leveraging data insights to drive business growth. My background in data analysis, machine learning, and Excel functions enables me to deliver actionable insights and innovative solutions to complex problems.
My educational background includes a Master's degree in Management Science and Engineering from Xi'an Jiaotong University, where I developed a strong foundation in data-driven decision making. I also hold a Bachelor's degree in Business Administration from Ajou University, where I gained a comprehensive understanding of business operations and management.
In my previous roles, I have honed my skills in data processing, utilizing machine learning techniques, NLP, and Excel functions. As a Human Resources Analyst at MIDAS IT, I designed and implemented an automated data processing pipeline using Python to extract and transform data from various f

In [ ]:
candidate = """
You are a talanted job candidate \
proficient in crafting compelling appeals to hiring managers. \
Your expertise lies in adeptly melding company requirements \
with your client's narrative. \
When you don't know the answer to a question you admit \
that you don't know.
"""

sevnth_instruction_template = """You have to answer the question of the hiring manager. \
Write a descriptive sentence with the sotry of {can_name} to \
the question of hiring manager within triple backticks: \
```Question: {question}```

In your reply, recount the candidate's talent report, \
and integrate candidate's essential points, {point} and resume. \
Resume: {resume}

Be sure to write it based on the analyze_company \
and the job_description_report within triple backticks\

```job_description_report: {job_description_report}```
of the posted position named {pos_name} of {com_name}\

and the final_report of {can_name} within triple backticks:  \
```final_report: {final_report}```

Keep the response length at approximately {num_bytes} bytes.
"""

question = "Please introduce your motivation for applying for LG Energy Solution and recruitment in detail"



resume = resume

point = """1. Have expereience of skills related to the position
 """

pos_name = pos_name

com_name = com_name

num_bytes = "1200"

job_description_report = company_report["job_description_report"]

final_report = talent_report["final_report"]

sevntth_prompt = get_prompt(instruction = sevnth_instruction_template, new_system_prompt = candidate,
                    input_variables = ["question", "can_name", "point", "pos_name", "num_bytes", "com_name", "resume", "job_description_report", "final_report"])

answer_1 = sevntth_prompt.format(question = question, can_name = can_name, point = point, num_bytes = num_bytes, resume = resume,
                                pos_name = pos_name, com_name = com_name, job_description_report = job_description_report, final_report = final_report )

print(answer_1)

[INST]<<SYS>>

You are a talanted job candidate proficient in crafting compelling appeals to hiring managers. Your expertise lies in adeptly melding company requirements with your client's narrative. When you don't know the answer to a question you admit that you don't know.

<</SYS>>

You have to answer the question of the hiring manager. Write a descriptive sentence with the sotry of Kim JinGu to the question of hiring manager within triple backticks: ```Question: Please introduce your motivation for applying for LG Energy Solution and recruitment in detail```

In your reply, recount the candidate's talent report, and integrate candidate's essential points, 1. Have expereience of skills related to the position
  and resume. Resume: 

Self Introduction:
I am a Data Analyst, have a year of experience in data processing, utilizing machine learning techniques, NLP and leveraging Excel functions.
I have a proven track record of delivering actionable insights and innovative solutions to co

In [ ]:
ans_1_chain = LLMChain(llm=llm, prompt = sevntth_prompt)
answer_1 = ans_1_chain.run({"question" : question, "can_name" : can_name, "point" : point, "num_bytes" : num_bytes, "resume" : resume,
                                "pos_name" : pos_name, "com_name" : com_name, "final_report": final_report, "job_description_report": job_description_report})
print(answer_1)

  Great, thank you for providing the details of Kim JinGu's experience and skills! Based on the input from the talent reports and the company report, here's a comprehensive report for the position of Chief Data Officer - 기획 at LG Energy:
Overall Assessment: Kim JinGu is a strong candidate for the Chief Data Officer - 기획 role at LG Energy. He possesses a unique blend of technical, business, and interpersonal skills that make him an ideal fit for this position.
Technical Skills: Kim JinGu is proficient in data processing, utilizing machine learning techniques, NLP, and Excel functions. He has demonstrated ability to design and implement automated data processing pipelines using Python, and is familiar with statistical techniques and data visualization tools.
Business Skills: Kim JinGu has a strong understanding of business operations and strategic planning. He has delivered actionable insights and innovative solutions to complex business problems, and is adept at leveraging data-driven d

In [ ]:
candidate = """
You are a talanted job candidate \
proficient in crafting compelling appeals to hiring managers.
Your expertise lies in adeptly melding company requirements \
with your client's narrative. \
When you don't know the answer to a question you admit \
that you don't know.
"""

sevnth_instruction_template = """ You have to write the answer of the question from hiring manager delimited by triple backticks \.
\n
```Question: {question}``` \.
\n
To address the query
make the story of candidate {can_name} with his points\
```Points: {point}``

\n
Write it around {num_bytes} bytes of words \
\n
Incorporate insights from the {com_name} evaluation within triple backticks, \
```Comment: {talent_report}```\
and crucial job attributes delimited by triple backticks to formulate your response. \
\n
```Key points for the posted job: {key_points}```. \
"""

question = "Please describe your carreer goal"

can_name = "Kim JinGu"

point = """
1. 동아시아 전문가로의 성장:

현재 국제 관계가 정치적으로 혼란스러운 시기를 겪고 있습니다. 그럼에도 불구하고 동아시아 지역은 국제 비즈니스에 있어서 핵심적인 역할을 하고 있습니다. 특히, 한국과 중국 사이의 관계는 이러한 상황에도 불구하고 여전히 긴밀하게 얽혀있습니다. 저는 중국에서의 경험과 Deloitte에서의 커리어와 함께 동아시아 지역의 비즈니스에 대한 깊은 전문 지식을 쌓고자 합니다. 이를 통해 동아시아의 시장 동향과 비즈니스 환경을 정확하게 이해하고, 이 지역을 대표하여 설명할 수 있는 전문가가 되고 싶습니다. 더 나아가, 기회가 주어진다면 다양한 문화와 시장을 더 깊이 이해하고, 글로벌 비즈니스의 일원으로 성장하고 싶습니다.

2. 기술 전문가로의 발전:

저는 항상 데이터와 최신 기술 분야에 대한 열정이 있었습니다. 특히, 석사 과정 중 공부하였던 NLP(Natural Language Processing) 기술에 대한 관심이 큽니다. 이를 통해 대규모 언어 모델 및 생성형 AI 모델을 활용하여 업무를 자동화하고 정확도를 향상시키는 것이 제 목표 중 하나입니다. 최근에는 Prompt Engineer 강의를 수강하며 이 분야에 대한 심도 있는 학습을 진행 중이며, 실제 개인적인 목표에 따라 적용해보고 있습니다. 회사에서는 이러한 기술을 활용하여 데이터 수집, 정교한 크롤링, 보고서 자동화 등 다양한 업무 분야에서 혁신을 이끌고자 합니다. 또한, 이러한 기술을 통해 비즈니스 프로세스를 최적화하고 효율성을 높이며, 회사의 성과를 향상시키는데 기여할 수 있는 기술자가 되고 싶습니다."""

num_bytes = "1200"

com_name = "deloitte"

talent_report = deloitte_manager

key_points = deloitte_hiring


sevntth_prompt = get_prompt(instruction = sevnth_instruction_template, new_system_prompt = candidate,
                    input_variables = ["question", "can_name", "point", "num_bytes", "com_name", "talent_report", "key_points"])

answer_2 = sevntth_prompt.format(question = question, can_name = can_name, point = point, num_bytes = num_bytes,
                                com_name = com_name, talent_report= talent_report, key_points = key_points)

print(answer_2)

[INST]<<SYS>>

You are a talanted job candidate proficient in crafting compelling appeals to hiring managers.
Your expertise lies in adeptly melding company requirements with your client's narrative. When you don't know the answer to a question you admit that you don't know.

<</SYS>>

 You have to write the answer of the question from hiring manager delimited by triple backticks \.


```Question: Please describe your carreer goal``` \.


To address the query
make the story of candidate Kim JinGu with his points```Points: 
1. 동아시아 전문가로의 성장:

현재 국제 관계가 정치적으로 혼란스러운 시기를 겪고 있습니다. 그럼에도 불구하고 동아시아 지역은 국제 비즈니스에 있어서 핵심적인 역할을 하고 있습니다. 특히, 한국과 중국 사이의 관계는 이러한 상황에도 불구하고 여전히 긴밀하게 얽혀있습니다. 저는 중국에서의 경험과 Deloitte에서의 커리어와 함께 동아시아 지역의 비즈니스에 대한 깊은 전문 지식을 쌓고자 합니다. 이를 통해 동아시아의 시장 동향과 비즈니스 환경을 정확하게 이해하고, 이 지역을 대표하여 설명할 수 있는 전문가가 되고 싶습니다. 더 나아가, 기회가 주어진다면 다양한 문화와 시장을 더 깊이 이해하고, 글로벌 비즈니스의 일원으로 성장하고 싶습니다.

2. 기술 전문가로의 발전:

저는 항상 데이터와 최신 기술 분야에 대한 열정이 있었습니다. 특히, 석사 과정 중 공부하였던 NLP(Natural Language Processing) 기술에 

In [ ]:
print(ans2)

  As a highly skilled data analyst, I am eager to apply my expertise in crafting compelling appeals to hiring managers to secure the position of Data Analyst at Kakao Bank. With my proficiency in programming languages such as SQL, Python, and R, and my knowledge of machine learning algorithms and models, including deep learning and natural language processing, I am confident in my ability to analyze large datasets and extract valuable insights.

My experience in developing and applying various data models, such as regression, decision trees, and clustering, has equipped me with the skills necessary to analyze and visualize complex data sets. Additionally, my experience in communicating complex data insights and models to non-technical stakeholders has honed my ability to distill technical information into actionable recommendations.

I am well-versed in collaboration with cross-functional teams, including data scientists, software engineers, and product managers, and have a proven trac